Installing Fake_useragent and Pandas

In [2]:
pip install fake_useragent pandas

  Using cached fake_useragent-2.0.3-py3-none-any.whl.metadata (17 kB)
  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)


  Using cached numpy-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached fake_useragent-2.0.3-py3-none-any.whl (201 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached numpy-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.1 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Importing all necessary libraries

In [3]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd

Setting up IMDb Scraper

In [4]:
url = 'https://www.imdb.com/chart/boxoffice/?ref_=nv_ch_cht'
ua = UserAgent()
userAgent = ua.random
headers = {'User-Agent': userAgent}
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, 'html.parser')

Scraping The top 10 movies, thier titles, Numbers and Rate votes

In [22]:
movie_box= soup.find_all('div', class_ = "ipc-metadata-list-summary-item__c")

titles = []
numbers = []
rate_votes =[]
for box in movie_box:

    if box.find('h3', class_ = "ipc-title__text") is not None:
            title = box.find('h3', class_= "ipc-title__text")
            titles.append(title.text)
    else:
        titles.append('None')

    if box.find('ul', attrs={'data-testid' : "title-metadata-box-office-data-container"}) is not None:
            number = box.find('ul', attrs={'data-testid' : "title-metadata-box-office-data-container"})
            numbers.append(number.text)
    else:
        numbers.append('None')

    if box.find('span', class_ = "ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating") is not None:
        rate_vote = box.find('span', class_ ="ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating")
        rate_votes.append(rate_vote.text.replace('\xa0', ' '))
    else:
        rate_votes.append('None')


rate_votes

['6.7 (33K)',
 '7.2 (1.6K)',
 '5.9 (6.9K)',
 '7.1 (31K)',
 '6.6 (4.8K)',
 '7.0 (67K)',
 '7.5 (101K)',
 '7.7 (17K)',
 '7.7 (111K)',
 '6.5 (17K)']

Saving them into a Panda DataFrame

In [51]:
movie_df = pd.DataFrame({
    'Titles':titles,
    'Numbers': numbers,
    'Rate_Votes': rate_votes
})
movie_df

,Titles,Numbers,Rate_Votes
0,1. Mufasa: The Lion King,Weekend Gross: $12MTotal Gross: $212MWeeks Rel...,6.7 (33K)
1,2. One of Them Days,Weekend Gross: $12MTotal Gross: $16MWeeks Rele...,7.2 (1.6K)
2,3. Wolf Man,Weekend Gross: $11MTotal Gross: $14MWeeks Rele...,5.9 (6.9K)
3,4. Sonic the Hedgehog 3,Weekend Gross: $8.6MTotal Gross: $220MWeeks Re...,7.1 (31K)
4,5. Den of Thieves 2: Pantera,Weekend Gross: $6.6MTotal Gross: $28MWeeks Rel...,6.6 (4.8K)
5,6. Moana 2,Weekend Gross: $6MTotal Gross: $445MWeeks Rele...,7.0 (67K)
6,7. Nosferatu,Weekend Gross: $4.1MTotal Gross: $91MWeeks Rel...,7.5 (101K)
7,8. A Complete Unknown,Weekend Gross: $3.7MTotal Gross: $59MWeeks Rel...,7.7 (17K)
8,9. Wicked: Part I,Weekend Gross: $3.6MTotal Gross: $466MWeeks Re...,7.7 (111K)
9,10. Babygirl,Weekend Gross: $2MTotal Gross: $26MWeeks Relea...,6.5 (17K)


Cleaning the dataframe and saving into a csv file

In [50]:
movie_df['Titles'] = movie_df['Titles'].str.replace(r'^\d+\.\s*', '', regex=True)
movie_df['Numbers'] = movie_df['Numbers'].str.replace('Weekend Gross:', '')
movie_df['Numbers'] = movie_df['Numbers'].str.replace('Total Gross: ', '|')
movie_df['Numbers'] = movie_df['Numbers'].str.replace('Weeks Released: ', '|')
movie_df[['Weekend Gross', 'Total Gross', 'Weeks Released']] = movie_df['Numbers'].str.split('|', expand=True)
movie_df[['Rating','Vote_count']] = movie_df['Rate_Votes'].str.split(' ', expand= True)
movie_df.drop(['Numbers', 'Rate_Votes'], axis=1, inplace=True)

movie_df.to_csv('Movie_Data.csv', index=False)
